[Home](http://realai.org/) > [Course](http://realai.org/course/) > [TensorFlow](http://realai.org/course/tensorflow/) > [Deep Models](http://realai.org/course/tensorflow/#deep-models) >

# Solving MNIST by Convolution

*Last Updated: August 27, 2017*

Same as in the [last session](http://realai.org/course/tensorflow/#classifying-handwritten-digits), we begin by importing TensorFlow and loading data into memory:

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
MNIST = input_data.read_data_sets("MNIST_data", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


Previously, we used `tf.layers.dense` to compute the logits. A model is considered "deep" when it has more layers. In this session, we will build a deep convolutional model in stages: first add another dense layer, next apply dropout to that layer, and finally add one or two convolution layers. The relevant code is in the next cell. As components are gradually added to the model, readers are encouraged to go through the entire session multiple times in order to get an intuitive sense of how additional layers affect model behavior. But the model's test set performance near the end should be saved only for the final run.
 
* Tip: Use a faster virtual machine instance when convolution layers are added!
 
The first step of going deep is to simply add another dense layer. The new layer must have a sensible activation function for non-linearity. Here we use [tf.nn.relu](https://www.tensorflow.org/api_docs/python/tf/nn/relu). Had we used `activation=None`, the two dense layers would be computationally equivalent to one dense layer whose weight matrix is the product of the two weight matrices in the original dense layers:
 
$W_2(W_1 X + B_1) + B_2 = (W_2 W_1)X + (W_2 B_1 + B_2).$
 
Using [tf.nn.dropout](https://www.tensorflow.org/api_docs/python/tf/nn/dropout), we can randomly keep a portion of the new dense layer's outputs, and set the rest to 0. This technique is called [dropout](https://en.wikipedia.org/wiki/Dropout_(neural_networks%29). Intuitively, it improves the overall quality of outputs because a random portion of the output must carry enough useful information for the downstream `Logits` layer to make the correct classification call.

Dense layers miss a crucial insight for our task: the input images are not just random 2D arrays of numbers that are labelled arbitrarily. Instead, they’re meaningful images in the world we live in. Because many physical laws are local, natural scenes tend to be composed of local patterns that are generic, such as angles, edges, curves, and various hierarchical combinations of them that form intermediate patterns. A network layer commonly used to parse these patterns is the convolution layer. A deep neural network prominently featuring convolution layers is often called a [Convolutional Neural Network](http://realai.org/course/convolutional-neural-network).

2D images are often represented as 3D arrays in computers, with the additional dimension representing channels. For example, a typical RGB color image has three channels, red, green, and blue. TensorFlow’s 2D convolution layer [tf.layer.conv2d](https://www.tensorflow.org/api_docs/python/tf/layers/conv2d) is designed for this type of inputs. Since our images are in grayscale, they have only one channel. We need to reshape the input tensor to `(-1, 28, 28, 1)`, where the last dimension `1` informs the downstream convolution layer that these inputs have exactly one channel. The output of the convolution layer is again a 4D tensor, conceptually like a batch of 2D images with one or more channels. They can be readily fed to a max pooling layer or another convolution layer for further processing. In this session we follow [Deep MNIST for Experts](https://www.tensorflow.org/get_started/mnist/pros), an introductory article on the official TensorFlow website, and use two convolution layers of 5x5 patches, with 32 features and 64 features, respectively. Either convolution layer is followed by a [tf.layers.max_pooling2d](https://www.tensorflow.org/api_docs/python/tf/layers/max_pooling2d) layer to pick the maximum output number for each non-overlapping 2x2 patch in each output channel. The max pooling layers simply downsample the upstream convolution layer output “images” to 50%. The first max pooling layer’s output is passed to the second convolution layer. The second max pooling layer’s output is “flattened” for downstream dense layers. Feel free to tweak the above parameters and see how they affect model performance.

The next cell of code builds the "deep" part of our model: Input -> Conv2d -> Max_Pooling -> Conv2d -> Max_Pooling -> Dense -> Logits, corresponding to all the steps needed for computing `Logits` in the graph below:

![](http://realai.org/course/tensorflow/solving-MNIST-by-convolution-1.png)

In [2]:
# Start with the original input
images = tf.placeholder(tf.float32, (None, 784), name="Images")

# Reshape the 1D images data to 2D for convolution
images_2d = tf.reshape(images, (-1, 28, 28, 1), name="Images_2D")

# Add two convolution layers with max pooling
conv1 = tf.layers.conv2d(images_2d, 32, 5, padding="same", name="Conv1")
pool1 = tf.layers.max_pooling2d(conv1, 2, 2, name="Pool1")
conv2 = tf.layers.conv2d(pool1, 64, 5, padding="same", name="Conv2")
pool2 = tf.layers.max_pooling2d(conv2, 2, 2, name="Pool2")

# Reshape the 2D tensor back to 1D to be fed into "Dense"
pool2_flat = tf.reshape(pool2, (-1, 7*7*64), name="Pool2_Flat")

# A dense layer with dropout
dense = tf.layers.dense(pool2_flat, 512, activation=tf.nn.relu, name="Dense")
keep_prob = tf.placeholder(tf.float32, name="Keep_Probability")
dropout = tf.nn.dropout(dense, keep_prob, name="Dropout")

# The original dense layer to compute logits that are later used for classification
logits = tf.layers.dense(dropout, 10, activation=None, name="Logits")

In [3]:
# We can take a look at the shapes of some tensors
(pool2.shape, pool2_flat.shape, dropout.shape, logits.shape)

(TensorShape([Dimension(None), Dimension(7), Dimension(7), Dimension(64)]),
 TensorShape([Dimension(None), Dimension(3136)]),
 TensorShape([Dimension(None), Dimension(512)]),
 TensorShape([Dimension(None), Dimension(10)]))

Loss and training remain the same as in the previous session:

In [4]:
labels = tf.placeholder(tf.float32, (None, 10), name="Labels")

with tf.name_scope("Loss"):
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=labels, logits=logits),
        name="Mean")

with tf.name_scope("Optimizer"):
    train = tf.train.AdamOptimizer(learning_rate=0.001, name="Adam").minimize(loss)

with tf.name_scope("Error"):
    error = tf.reduce_mean(
        tf.cast(tf.not_equal(tf.argmax(labels, 1), tf.argmax(logits, 1)), tf.float32), name="Mean")

The 2D image images can be directly displayed in TensorBoard:

In [5]:
tf.summary.image("Images_2D", images_2d, max_outputs=4)
tf.summary.scalar("Loss", loss)
tf.summary.scalar("Error", error)
summ = tf.summary.merge_all()

Session, FileWriter and variable initialization:

In [6]:
LOGDIR = "/tmp/MNIST_conv"
if tf.gfile.Exists(LOGDIR):
    tf.gfile.DeleteRecursively(LOGDIR)
tf.gfile.MakeDirs(LOGDIR)

sess = tf.Session()
writer = tf.summary.FileWriter(LOGDIR, sess.graph)
sess.run(tf.global_variables_initializer())

Convolution needs a lot of computing power. We can use a cell [magic function](https://ipython.readthedocs.io/en/stable/interactive/tutorial.html#magics-explained) `%%time` to measure how long it takes to run the code in a cell. Training the full model on an [n1-highcpu-8](https://cloud.google.com/compute/pricing#predefined_machine_types) (8 vCPUs, 7.2 GB memory) should take less than 20 minutes:

In [7]:
%%time
for i in range(10000):
    batch = MNIST.train.next_batch(64)
    
    if i % 10 == 0:
        Error, Loss, Summ = sess.run((error, loss, summ), feed_dict={images: batch[0], labels: batch[1], keep_prob: 1.0})
        writer.add_summary(Summ, i)
        
        if i % 500 == 0:
            print("Step {}: Training loss is {:.5f}, error is {:.2f}%".format(i, Loss, Error * 100))

    sess.run(train, feed_dict={images: batch[0], labels: batch[1], keep_prob: 0.5})

Step 0: Training loss is 2.30561, error is 85.94%
Step 500: Training loss is 0.22809, error is 3.12%
Step 1000: Training loss is 0.02456, error is 1.56%
Step 1500: Training loss is 0.09726, error is 3.12%
Step 2000: Training loss is 0.00252, error is 0.00%
Step 2500: Training loss is 0.02458, error is 1.56%
Step 3000: Training loss is 0.00108, error is 0.00%
Step 3500: Training loss is 0.04434, error is 1.56%
Step 4000: Training loss is 0.00234, error is 0.00%
Step 4500: Training loss is 0.00004, error is 0.00%
Step 5000: Training loss is 0.00435, error is 0.00%
Step 5500: Training loss is 0.00016, error is 0.00%
Step 6000: Training loss is 0.02235, error is 1.56%
Step 6500: Training loss is 0.00006, error is 0.00%
Step 7000: Training loss is 0.00066, error is 0.00%
Step 7500: Training loss is 0.15628, error is 4.69%
Step 8000: Training loss is 0.00272, error is 0.00%
Step 8500: Training loss is 0.00231, error is 0.00%
Step 9000: Training loss is 0.00143, error is 0.00%
Step 9500: Trai

On a virtual machine with a GPU, the above training would only take less than 2 minutes! The GPU VM we used also had 4 CPUs, but clearly their contribution was just a rounding error. In a [July 2017 benchmarking exercise](http://minimaxir.com/2017/07/cpu-or-gpu/), a regular GPU performs better than 64 ([preemptible](https://cloud.google.com/compute/docs/instances/preemptible)) vCPUs, at similar costs. For CNNs larger than the toy model used in this session, readers are advised to build a GPU VM.
 
Print the validation result:

In [8]:
Error = sess.run(error, feed_dict={images: MNIST.validation.images, labels: MNIST.validation.labels, keep_prob: 1.0})
print("Validation error is {:.2f}%".format(Error * 100))

Validation error is 1.02%


If you've run a lot of different model variations, you might be tempted to pick the best-performing model and end up with an artificially low validation error. This is known as [overfitting](https://en.wikipedia.org/wiki/Overfitting), when a lucky result is presented as the generalizable performance of a model. Test data should be used only once to more reliably measure the model's true error rate:

In [9]:
Error = sess.run(error, feed_dict={images: MNIST.test.images, labels: MNIST.test.labels, keep_prob: 1.0})
print("Test error is {:.2f}%".format(Error * 100))

Test error is 1.22%


Our test error and validation error are fairly close. An error rate of 1.22% is still way above what the best models today can achieve, but at least it is presentable in an online [collection](https://rodrigob.github.io/are_we_there_yet/build/classification_datasets_results#4d4e495354) of reported results, not bad for beginners!

Close FileWriter and Session:

In [10]:
writer.close()
sess.close()